In [6]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import timm
from PIL import Image

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3 channels
    transforms.Resize((224, 224)),               # Resize for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize
])

# Dataset loading
mnist_dataset = MNIST(root="./data", train=True, transform=transform, download=True)
subset_size = 5000  # Use a subset of data for faster training
mnist_dataset = torch.utils.data.random_split(mnist_dataset, [subset_size, len(mnist_dataset) - subset_size])[0]

# Splitting into training and validation sets
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(mnist_dataset, test_size=0.1, random_state=42)

# Data loaders
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4)

# Vision Transformer Model
class ViT(nn.Module):
    def __init__(self, num_classes=10):
        super(ViT, self).__init__()
        self.model = timm.create_model("vit_small_patch16_224", pretrained=True)
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

model = ViT().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

# Training loop
num_epochs = 5
accumulation_steps = 4

for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")

print("Training complete!")



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.51MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.68MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1/5, Loss: 0.3484, Accuracy: 0.9300
Epoch 2/5, Loss: 0.0498, Accuracy: 0.9520
Epoch 3/5, Loss: 0.4081, Accuracy: 0.9560
Epoch 4/5, Loss: 0.0250, Accuracy: 0.9280
Epoch 5/5, Loss: 0.0814, Accuracy: 0.9600
Training complete!


In [7]:
# Define MNIST class labels
mnist_classes = [str(i) for i in range(10)]

# Function to classify a single image
def classify_image(model, image_path, transform, class_labels):
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    image = image.convert("RGB")  # Convert to 3 channels (for ViT compatibility)
    image = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)

    _, predicted_class_index = torch.max(output, 1)
    predicted_class_label = class_labels[predicted_class_index.item()]

    return predicted_class_label

# Example image paths
image_path1 = "download.jpg"
predicted_class_label1 = classify_image(model, image_path1, transform, mnist_classes)
print(f"The predicted class label for the image is: {predicted_class_label1}")


The predicted class label for the image is: 9
